In [3]:
# 스파크 객체 생성
import pyspark
import pyspark.sql

sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc).builder.getOrCreate()


In [4]:
# 줄이지 않은 Raw 데이터 가져오기
rawdata = spark.read.parquet('../../02_Data_Batch_Processing/data/Raw_Data.parquet')
# 테이블 등록
rawdata.registerTempTable("rawdata")

In [5]:
# 월별 항공편 수
total_flights_by_month = spark.sql(
  """SELECT INT(Month), INT(Year), COUNT(*) AS total_flights
  FROM rawdata
  GROUP BY INT(Year), INT(Month)
  ORDER BY INT(Year), INT(Month)"""
)

total_flights_by_month.show()

+-----+----+-------------+
|Month|Year|total_flights|
+-----+----+-------------+
|    1|2015|       469968|
|    2|2015|       429191|
|    3|2015|       504312|
|    4|2015|       485151|
|    5|2015|       496993|
|    6|2015|       503897|
|    7|2015|       520718|
|    8|2015|       510536|
|    9|2015|       464946|
|   10|2015|       486165|
|   11|2015|       467972|
|   12|2015|       479230|
+-----+----+-------------+



In [6]:
# 모든 항공편에 운영된 고유한 항공기 수
tail_numbers = spark.sql(
  """SELECT count(distinct TailNum)
  FROM rawdata"""
)

tail_numbers.show()

+-----------------------+
|count(DISTINCT TailNum)|
+-----------------------+
|                   4897|
+-----------------------+



In [7]:
# 각 항공사가 보유한 항공기 개수 및 비율
carrier_tail_numbers = spark.sql(
  """SELECT Carrier, count(distinct TailNum) as tail_count, ROUND((count(distinct TailNum)/(SELECT count(distinct TailNum)
  FROM rawdata))*100,2) as tail_share
  FROM rawdata
  GROUP BY Carrier
  ORDER BY count(distinct TailNum) DESC
  """
)

carrier_tail_numbers.show()

+-------+----------+----------+
|Carrier|tail_count|tail_share|
+-------+----------+----------+
|     AA|      1044|     21.32|
|     DL|       828|     16.91|
|     UA|       721|     14.72|
|     WN|       704|     14.38|
|     EV|       390|      7.96|
|     OO|       383|      7.82|
|     US|       351|      7.17|
|     B6|       215|      4.39|
|     MQ|       203|      4.15|
|     AS|       147|       3.0|
|     NK|        79|      1.61|
|     F9|        62|      1.27|
|     VX|        57|      1.16|
|     HA|        50|      1.02|
+-------+----------+----------+



In [10]:
# 데이터 가져오기
on_time_dataframe = rawdata

# 전체 항공편 수 구하기
total_flights = on_time_dataframe.count()

# 출발이 늦은 항공편 수 구하기
late_departures = on_time_dataframe.filter(on_time_dataframe.DepDelayMinutes > 0)
total_late_departures = late_departures.count()

# 도착이 늦은 항공편 수 구하기
late_arrivals = on_time_dataframe.filter(on_time_dataframe.ArrDelayMinutes > 0)
total_late_arrivals = late_arrivals.count()

# 출발은 늦었지만 정시 도착한 항공편 수 구하기
on_time_heros = on_time_dataframe.filter(
  (on_time_dataframe.DepDelayMinutes > 0)
  &
  (on_time_dataframe.ArrDelayMinutes <= 0)
)
total_on_time_heros = on_time_heros.count()

# 전체 항공편 중 늦게 도착한 항공편 수 구하기
pct_late = round((total_late_arrivals / (total_flights * 1.0)) * 100, 1)

print("전체 항공 수:   {:,}".format(total_flights))
print("Late departures: {:,}".format(total_late_departures))
print("Late arrivals:   {:,}".format(total_late_arrivals))
print("Recoveries:      {:,}".format(total_on_time_heros))
print("Percentage Late: {}%".format(pct_late))

# 전체 항공편중 얼마나 많이 늦었는지 통계
# 전체 항공편 :약 600만
# 출발이 늦은 : 약 200만
# 도착이 늦은 : 약 200만
# 출발은 늦었지만 정시 도착한 : 약 60만
# 전체 항공편 중 늦게 도착한 항공기: 200만/600만 약 35%

전체 항공 수:   5,819,079
Late departures: 2,125,618
Late arrivals:   2,086,896
Recoveries:      606,902
Percentage Late: 35.9%


In [11]:
# 평균 출발지연, 평균 도착 지연
spark.sql("""
SELECT
  ROUND(AVG(DepDelay),1) AS AvgDepDelay,
  ROUND(AVG(ArrDelay),1) AS AvgArrDelay
FROM rawdata
"""
).show()

+-----------+-----------+
|AvgDepDelay|AvgArrDelay|
+-----------+-----------+
|        9.4|        4.4|
+-----------+-----------+



In [13]:
# 도착 지연원인과 걸린 시간
late_flights = spark.sql("""
SELECT
  ArrDelayMinutes,
  WeatherDelay,
  CarrierDelay,
  NASDelay,
  SecurityDelay,
  LateAircraftDelay
FROM
  rawdata
WHERE
  WeatherDelay IS NOT NULL
  OR
  CarrierDelay IS NOT NULL
  OR
  NASDelay IS NOT NULL
  OR
  SecurityDelay IS NOT NULL
  OR
  LateAircraftDelay IS NOT NULL
ORDER BY
  FlightDate
""")
late_flights.sample(False, 0.01).show()

+---------------+------------+------------+--------+-------------+-----------------+
|ArrDelayMinutes|WeatherDelay|CarrierDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+---------------+------------+------------+--------+-------------+-----------------+
|           19.0|         0.0|         1.0|     0.0|          0.0|             18.0|
|           25.0|         0.0|         0.0|    25.0|          0.0|              0.0|
|           30.0|         0.0|        13.0|     0.0|          0.0|             17.0|
|           60.0|         0.0|        43.0|    17.0|          0.0|              0.0|
|           58.0|         0.0|         0.0|     0.0|          0.0|             58.0|
|           20.0|         0.0|         4.0|     0.0|          0.0|             16.0|
|          192.0|         0.0|         0.0|   164.0|          0.0|             28.0|
|           67.0|         0.0|        11.0|     0.0|          0.0|             56.0|
|           28.0|         0.0|         6.0|    21.0|          0.0

In [14]:
# 도착 지연원인과 평균 걸린 시간
total_delays = spark.sql("""
SELECT
  ROUND(SUM(WeatherDelay)/SUM(ArrDelayMinutes) * 100, 1) AS pct_weather_delay,
  ROUND(SUM(CarrierDelay)/SUM(ArrDelayMinutes) * 100, 1) AS pct_carrier_delay,
  ROUND(SUM(NASDelay)/SUM(ArrDelayMinutes) * 100, 1) AS pct_nas_delay,
  ROUND(SUM(SecurityDelay)/SUM(ArrDelayMinutes) * 100, 1) AS pct_security_delay,
  ROUND(SUM(LateAircraftDelay)/SUM(ArrDelayMinutes) * 100, 1) AS pct_late_aircraft_delay
FROM rawdata
""")
total_delays.show()

+-----------------+-----------------+-------------+------------------+-----------------------+
|pct_weather_delay|pct_carrier_delay|pct_nas_delay|pct_security_delay|pct_late_aircraft_delay|
+-----------------+-----------------+-------------+------------------+-----------------------+
|              4.5|             29.2|         20.7|               0.1|                   36.1|
+-----------------+-----------------+-------------+------------------+-----------------------+



In [15]:
# 예정된 출발과 도착 시간
simple_on_time_features = spark.sql("""
    SELECT
    FlightDate,
      CRSDepTime,
      CRSArrTime
    FROM rawdata
    """)
simple_on_time_features.show(20)

+----------+----------+----------+
|FlightDate|CRSDepTime|CRSArrTime|
+----------+----------+----------+
|2015-05-09|      0550|      0835|
|2015-05-09|      1930|      2200|
|2015-05-09|      1100|      1635|
|2015-05-09|      1730|      1830|
|2015-05-09|      0620|      1000|
|2015-05-09|      2030|      2140|
|2015-05-09|      1120|      1405|
|2015-05-09|      0645|      1025|
|2015-05-09|      1500|      1920|
|2015-05-09|      1550|      2255|
|2015-05-09|      1410|      1505|
|2015-05-09|      1135|      1320|
|2015-05-09|      0740|      1050|
|2015-05-09|      0725|      1040|
|2015-05-09|      1130|      1435|
|2015-05-09|      1520|      1800|
|2015-05-09|      1850|      2130|
|2015-05-09|      1105|      1340|
|2015-05-09|      0640|      1005|
|2015-05-09|      1620|      1730|
+----------+----------+----------+
only showing top 20 rows



# 몽고DB에 거리 데이터 넣기

In [ ]:
import pyspark
import pyspark.sql
import sys, os, re

# 위에서 실행했다면 여기서 안해도 됨.. sparkcontext는 1번
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc).builder.getOrCreate()

In [1]:
rawdata = spark.read.parquet('../data/Raw_Data.parquet')
rawdata.registerTempTable("rawdata")

In [2]:
# 공항 출발지, 도착지 이름과 거리
# !! 같은 경로라도 거리가 다른 것들이 있어서 평균으로 처리해서 하나의 값만 남김.
origin_dest_distances = spark.sql("""
  SELECT Origin, Dest, AVG(Distance) AS Distance
  FROM rawdata
  GROUP BY Origin, Dest
  ORDER BY Distance DESC
""")
origin_dest_distances.show(5)
origin_dest_distances.count()

+------+----+--------+
|Origin|Dest|Distance|
+------+----+--------+
|   HNL| JFK|  4983.0|
|   JFK| HNL|  4983.0|
|   HNL| EWR|  4962.0|
|   EWR| HNL|  4962.0|
|   IAD| HNL|  4817.0|
+------+----+--------+
only showing top 5 rows



4696

In [3]:
# jsonl 파일로 만들기
origin_dest_distances.repartition(1).write.mode("overwrite").json("../data/origin_dest_distances.json")
os.system("rm ../data/origin_dest_distances.jsonl")
os.system("cat ../data/origin_dest_distances.json/part* > ../data/origin_dest_distances.jsonl")

0

In [5]:
!head -5 ../data/origin_dest_distances.jsonl

{"Origin":"JFK","Dest":"HNL","Distance":4983.0}
{"Origin":"HNL","Dest":"JFK","Distance":4983.0}
{"Origin":"HNL","Dest":"EWR","Distance":4962.0}
{"Origin":"EWR","Dest":"HNL","Distance":4962.0}
{"Origin":"IAD","Dest":"HNL","Distance":4817.0}


In [1]:
# 먼저 mongoimoport 설치 해야됨
# sudo apt-get install mongo-tools

# 몽고DB에 데이터 넣기
#!mongoimport --host mongodbcluster-shard-00-00-mhvk4.mongodb.net:27017 --db test --collection test --type json --file ../data/origin_dest_distances.jsonl --authenticationDatabase admin --ssl --username jang --password jang
!mongoimport --host ac-yquygh3-shard-00-01.u70vq2s.mongodb.net:27017 --db test --collection Distance --type json --file ../data/origin_dest_distances.jsonl --authenticationDatabase admin --ssl --username jang --password jang    

2022-07-18T02:10:00.708+0000	connected to: ac-yquygh3-shard-00-01.u70vq2s.mongodb.net:27017
2022-07-18T02:10:02.337+0000	imported 4696 documents
